In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pickle

from moviepy.editor import VideoFileClip

<IPython.core.display.Javascript object>

In [3]:
def create_output_directory(fpath):
    """
    Creates output directory for images, if it doesn't already exist
    """

    pathlib.Path(fpath).mkdir(parents=True, exist_ok=True)

<IPython.core.display.Javascript object>

In [4]:
test_images_input_dir = "./test_images/"
camera_calibration_input_dir = "./camera_cal/"

output_images_dir = "./output_images/"
output_videos_dir = "./output_videos/"
output_data_dir = "./output_data/"

camera_calibration_chessboard_corners_output_dir = (
    output_images_dir + "1_camera_calibration_output_images/chessboard_corners/"
)
camera_calibration_undistorted_output_dir = (
    output_images_dir + "1_camera_calibration_output_images/undistorted/"
)
perspective_transform_output_dir = (
    output_images_dir + "2_perspective_transform_output_images/"
)
color_threshold_output_dir = output_images_dir + "3_color_threshold_output_images/"
lane_lines_img_output_dir = output_images_dir + "4_lane_lines_output_images/"

<IPython.core.display.Javascript object>

In [5]:
create_output_directory(output_data_dir)
create_output_directory(camera_calibration_chessboard_corners_output_dir)
create_output_directory(camera_calibration_undistorted_output_dir)
create_output_directory(perspective_transform_output_dir)
create_output_directory(color_threshold_output_dir)
create_output_directory(lane_lines_img_output_dir)

<IPython.core.display.Javascript object>

# Calibrate Camera

- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html
- images in `camera_cal/` have chessboard size $9x6$

In [6]:
chess_w = 9
chess_h = 6

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ... (6, 4, 0)
# by creating six by eight points in an array
# each with three columns for the x, y, and z coordinates of each corner
# initialize these all as zeros using Numpy's zero function
objp = np.zeros((chess_w * chess_h, 3), np.float32)

# z coordinate will stay zero but for first two columns, x and y
# use numpy's mgrid function to generate the coordinates that we want
# mgrid returns the coordinate values for a given grid size
# shape those coordinates back into two columns, one for x and one for y
objp[:, :2] = np.mgrid[0:chess_w, 0:chess_h].T.reshape(-1, 2)  # x, y coordinates

# Arrays to store object points and image points from all the images
objpoints = []  # 3D points in real world space
imgpoints = []  # 2D points in image plane

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in os.listdir(camera_calibration_input_dir):
    # Read in each image
    cam_cal_img = cv2.imread(camera_calibration_input_dir + fname)

    # Convert to grayscale
    cam_cal_gray_img = cv2.cvtColor(cam_cal_img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(cam_cal_gray_img, (chess_w, chess_h), None)

    # If corners are found, add object points, image points
    if ret == True:
        objpoints.append(
            objp
        )  # these object points will be the same for all of the calibration images, since they represent a real chessboard

        # increase corner accuracy
        corners2 = cv2.cornerSubPix(
            cam_cal_gray_img, corners, (11, 11), (-1, -1), criteria
        )
        imgpoints.append(corners2)

        # Draw and display the corners
        cam_cal_img = cv2.drawChessboardCorners(
            cam_cal_img, (chess_w, chess_h), corners, ret
        )

        # plt.figure()
        # plt.imshow(cam_cal_img)

        # save image
        cv2.imwrite(
            camera_calibration_chessboard_corners_output_dir + fname, cam_cal_img
        )

<IPython.core.display.Javascript object>

In [7]:
# Load camera calibration example image
cam_cal_example_img = cv2.imread("./camera_cal/calibration1.jpg")
cam_cal_example_img_size = (cam_cal_example_img.shape[1], cam_cal_example_img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, cam_cal_example_img_size, None, None
)

<IPython.core.display.Javascript object>

In [8]:
# save calculated camera matrix and distortion coefficients
calibrate_camera_data = {"mtx": mtx, "dist": dist}

# store data (serialize)
with open(output_data_dir + "calibrate_camera_data.pickle", "wb") as handle:
    pickle.dump(calibrate_camera_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

<IPython.core.display.Javascript object>

# Perspective Transform

In [9]:
# Four source coordinates (region of interest)
perspective_src = np.float32(
    [
        (575, 465),  # top left
        (710, 465),  # top right
        (255, 685),  # bottom left
        (1050, 685),  # bottom right
    ]
)

y_size = 720  # height
x_size = 1280  # width

# Four desired coordinates (region of interest)
perspective_dst = np.float32(
    [
        (450, 0),  # top left
        (x_size - 450, 0),  # top right
        (450, y_size),  # bottom left
        (x_size - 450, y_size),  # bottom right
    ]
)

<IPython.core.display.Javascript object>

In [10]:
# Define perspective transform function
def perspective_transform(img_input, src, dst):
    """
    Applies perspective transform on input image based on src and dst parameters.
    """

    # Define calibration box in source (original) and destination (desired or warped) coordinates
    img_size = (img_input.shape[1], img_input.shape[0])

    # Compute the perspective transform, M (matrix)
    M = cv2.getPerspectiveTransform(src, dst)

    # Compute the inverse also by swapping the input parameters
    M_inv = cv2.getPerspectiveTransform(dst, src)

    # Create warped image using the perspective transform, M - uses linear interpolation
    img_warped = cv2.warpPerspective(img_input, M, img_size, flags=cv2.INTER_LINEAR)

    return img_warped, M_inv

<IPython.core.display.Javascript object>

# Color Spaces and Thresholding

In [11]:
# threshold the L-channel of HLS
def hls_l_threshold(img_input, thresh=(220, 255)):
    """
    Extracts L channel of HLS colorspace.
    Uses default threshold values if not specified explicitly.    
    """

    # convert to HLS color space
    hls_l_channel = cv2.cvtColor(img_input, cv2.COLOR_RGB2HLS)[:, :, 1]
    hls_l_channel = hls_l_channel * (255 / np.max(hls_l_channel))

    # apply a threshold to the L channel
    hls_l_binary = np.zeros_like(hls_l_channel)
    hls_l_binary[(hls_l_channel > thresh[0]) & (hls_l_channel <= thresh[1])] = 1

    return hls_l_binary


# threshold the B-channel of LAB
def lab_b_threshold(img_input, thresh=(190, 255)):
    """
    Extracts B channel of LAB colorspace.
    Uses default threshold values if not specified explicitly.    
    """

    # convert to LAB color space
    lab_b_channel = cv2.cvtColor(img_input, cv2.COLOR_RGB2Lab)[:, :, 2]
    # don't normalize if there are no yellows in the image
    if np.max(lab_b_channel) > 175:
        lab_b_channel = lab_b_channel * (255 / np.max(lab_b_channel))

    # apply a threshold to the B channel
    lab_b_binary = np.zeros_like(lab_b_channel)
    lab_b_binary[((lab_b_channel > thresh[0]) & (lab_b_channel <= thresh[1]))] = 1

    return lab_b_binary

<IPython.core.display.Javascript object>

# Sliding Window

In [12]:
def sliding_window_polyfit(img_input):
    """
    Applies sliding window algorithm on input image to extract lane lines.
    """

    # Take a histogram of the bottom half of the image
    histogram = np.sum(img_input[img_input.shape[0] // 2 :, :], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] // 2)
    quarter_point = np.int(midpoint // 2)
    leftx_base = np.argmax(histogram[quarter_point:midpoint]) + quarter_point
    rightx_base = np.argmax(histogram[midpoint : (midpoint + quarter_point)]) + midpoint

    # HYPERPARAMETERS - related to our sliding windows
    # Choose the number of sliding windows
    nwindows = 10
    # Set the width of the windows +/- margin
    margin = 90
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img_input.shape[0] // nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img_input.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Rectangles data for sliding windows visualization
    rectangles_visualization_data = []

    # 1. Loop through each window in nwindows

    # 2. Find the boundaries of our current window.
    #    This is based on a combination of the current window's starting point
    #    (leftx_current and rightx_current), as well as the margin you set in the hyperparameters.

    # 3. Use cv2.rectangle to draw these window boundaries onto our visualization image out_img.
    #    This is required for the quiz, but you can skip this step in practice if you don't need
    #    to visualize where the windows are.

    # 4. Now that we know the boundaries of our window, find out which activated pixels from
    #    nonzeroy and nonzerox above actually fall into the window.

    # 5. Append these to our lists left_lane_inds and right_lane_inds.

    # 6. If the number of pixels you found in Step 4 are greater than your hyperparameter minpix,
    #    re-center our window (i.e. leftx_current or rightx_current) based on the mean position of these pixels.

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img_input.shape[0] - (window + 1) * window_height
        win_y_high = img_input.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = (
            (nonzeroy >= win_y_low)
            & (nonzeroy < win_y_high)
            & (nonzerox >= win_xleft_low)
            & (nonzerox < win_xleft_high)
        ).nonzero()[0]
        good_right_inds = (
            (nonzeroy >= win_y_low)
            & (nonzeroy < win_y_high)
            & (nonzerox >= win_xright_low)
            & (nonzerox < win_xright_high)
        ).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fit, right_fit = (None, None)
    # Fit a second order polynomial to each side/lane
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)

    return (left_fit, right_fit, left_lane_inds, right_lane_inds)

<IPython.core.display.Javascript object>

# Search from Prior

In [13]:
def search_around_poly(img_input, left_fit_prev, right_fit_prev, margin=80):
    """
    Searches lane lines in a margin around the previous lane line position.
    """
    # margin parameter is the width of the margin around the previous polynomial to search

    # Grab activated pixels
    nonzero = img_input.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = (
        nonzerox
        > (
            left_fit_prev[0] * (nonzeroy ** 2)
            + left_fit_prev[1] * nonzeroy
            + left_fit_prev[2]
            - margin
        )
    ) & (
        nonzerox
        < (
            left_fit_prev[0] * (nonzeroy ** 2)
            + left_fit_prev[1] * nonzeroy
            + left_fit_prev[2]
            + margin
        )
    )
    right_lane_inds = (
        nonzerox
        > (
            right_fit_prev[0] * (nonzeroy ** 2)
            + right_fit_prev[1] * nonzeroy
            + right_fit_prev[2]
            - margin
        )
    ) & (
        nonzerox
        < (
            right_fit_prev[0] * (nonzeroy ** 2)
            + right_fit_prev[1] * nonzeroy
            + right_fit_prev[2]
            + margin
        )
    )

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fit, right_fit = (None, None)
    # Fit a second order polynomial to each side/lane
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit, left_lane_inds, right_lane_inds

<IPython.core.display.Javascript object>

# Calculate lane curvature

In [14]:
def calculate_lane_curvature(
    img_binary, left_lane_inds, right_lane_inds, ym_per_pix, xm_per_pix
):
    """
    Calculates the curvature of polynomial functions in meters.
    """
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    ploty = np.linspace(0, img_binary.shape[0] - 1, img_binary.shape[0])
    y_eval = np.max(ploty)

    # Grab activated pixels
    nonzero = img_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_curverad, right_curverad = 0, 0

    if len(leftx) != 0 and len(rightx) != 0:
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)

        # Calculate the new radius of curvature in meters
        left_curverad = (
            (1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1]) ** 2)
            ** 1.5
        ) / np.absolute(2 * left_fit_cr[0])
        right_curverad = (
            (1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1]) ** 2)
            ** 1.5
        ) / np.absolute(2 * right_fit_cr[0])

    return left_curverad, right_curverad

<IPython.core.display.Javascript object>

# Calculate distance from center

In [15]:
def calculate_distance_from_center(img_binary, left_fit, right_fit, xm_per_pix):
    """
    Calculates vehicle's distance from lane center.
    """

    # Distance from center is image x-axis midpoint
    # we use mean of left_fit and right_fit intercepts
    if left_fit is not None and right_fit is not None:
        car_position = img_binary.shape[1] / 2

        left_fit_x_int = (
            left_fit[0] * img_binary.shape[0] ** 2
            + left_fit[1] * img_binary.shape[0]
            + left_fit[2]
        )
        right_fit_x_int = (
            right_fit[0] * img_binary.shape[0] ** 2
            + right_fit[1] * img_binary.shape[0]
            + right_fit[2]
        )
        lane_center_position = (left_fit_x_int + right_fit_x_int) / 2

    # return distance from center
    return (car_position - lane_center_position) * xm_per_pix

<IPython.core.display.Javascript object>

# Draw lane

In [16]:
def draw_lane(original_img, binary_img, left_fit, right_fit, Minv):
    new_img = np.copy(original_img)
    if left_fit is None or right_fit is None:
        return original_img

    # create an image to draw the lines on
    warp_zero = np.zeros_like(binary_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    ploty = np.linspace(0, binary_img.shape[0] - 1, num=binary_img.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(
        color_warp, Minv, (binary_img.shape[1], binary_img.shape[0])
    )
    # Combine the result with the original image
    result = cv2.addWeighted(new_img, 1, newwarp, 0.5, 0)

    return result

<IPython.core.display.Javascript object>

# Draw data

In [17]:
def draw_data(original_img, curve_radius, distance_from_center):
    output_img = np.copy(original_img)
    cv2.putText(
        output_img,
        "Curve radius: " + "{:04.2f}".format(curve_radius) + "m",
        (40, 80),
        cv2.FONT_HERSHEY_DUPLEX,
        1,
        (152, 251, 152),
        2,
        cv2.LINE_AA,
    )

    direction = ""
    if distance_from_center > 0:
        direction = "right"
    elif distance_from_center < 0:
        direction = "left"
    cv2.putText(
        output_img,
        "{:04.3f}".format(abs(distance_from_center)) + "m " + direction + " of center",
        (40, 120),
        cv2.FONT_HERSHEY_DUPLEX,
        1,
        (152, 251, 152),
        2,
        cv2.LINE_AA,
    )

    return output_img

<IPython.core.display.Javascript object>

# Define a class for storing data

In [18]:
class Line:
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False

        # x values of the last n fits of the line
        self.recent_xfitted = []

        # average x values of the fitted line over the last n iterations
        self.bestx = None

        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None

        # polynomial coefficients for the most recent fit
        self.current_fit = []

        # radius of curvature of the line in some units
        self.radius_of_curvature = None

        # distance in meters of vehicle center from the line
        self.line_base_pos = None

        # difference in fit coefficients between last and new fits
        self.diffs = np.array([0, 0, 0], dtype="float")

        # number of detected pixels
        self.px_count = None

    def add_fit(self, new_fit, inds):
        # add a found fit to the line, up to n
        if new_fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, compare it with a new fit
                self.diffs = abs(new_fit - self.best_fit)
            if (
                self.diffs[0] > 0.001 or self.diffs[1] > 1.0 or self.diffs[2] > 100.0
            ) and len(self.current_fit) > 0:
                # dismiss a bad fit, unless there are no fits in the current_fit queue
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(inds)
                self.current_fit.append(new_fit)
                if len(self.current_fit) > 5:
                    # throw out old fits, keep newest 5
                    self.current_fit = self.current_fit[len(self.current_fit) - 5 :]
                self.best_fit = np.average(self.current_fit, axis=0)
        # remove one fit from the history, if not found
        else:
            self.detected = False
            if len(self.current_fit) > 0:
                # removeoldest fit
                self.current_fit = self.current_fit[: len(self.current_fit) - 1]
            if len(self.current_fit) > 0:
                # if there are still any fits in the queue, best_fit is their average
                self.best_fit = np.average(self.current_fit, axis=0)

<IPython.core.display.Javascript object>

# Define pipeline

In [19]:
def lane_finding_pipeline(img_input, left_line, right_line, fname=None):
    new_img = np.copy(img_input)

    # undistort image
    undist_img = cv2.undistort(new_img, mtx, dist, None, mtx)

    # perspective transform
    warped_img, M_inv = perspective_transform(
        undist_img, perspective_src, perspective_dst
    )

    #  apply color threshold
    hls_l_binary = hls_l_threshold(warped_img)
    lab_b_binary = lab_b_threshold(warped_img)
    color_threshold_img = np.zeros_like(lab_b_binary)
    color_threshold_img[(hls_l_binary == 1) | (lab_b_binary == 1)] = 1

    # if one or both lane lines were not detected from last frame, use sliding_window_polyfit
    # otherwise, use search_around_poly
    if not left_line.detected or not right_line.detected:
        left_fit, right_fit, left_lane_inds, right_lane_inds, = sliding_window_polyfit(
            color_threshold_img
        )
    else:
        left_fit, right_fit, left_lane_inds, right_lane_inds = search_around_poly(
            color_threshold_img, left_line.best_fit, right_line.best_fit
        )

    left_line.add_fit(left_fit, left_lane_inds)
    right_line.add_fit(right_fit, right_lane_inds)

    # Define conversions in x and y from pixels space to meters
    # lane length is 10 ft = 3.048 meters
    # lane width is 12 ft = 3.6576 meters
    # distance between lanes is 380 pixels
    ym_per_pix = 3.048 / 100  # meters per pixel in y dimension
    xm_per_pix = 3.6576 / 380  # meters per pixel in x dimension

    radius_left, radius_right = calculate_lane_curvature(
        color_threshold_img, left_lane_inds, right_lane_inds, ym_per_pix, xm_per_pix
    )

    distance_from_center = calculate_distance_from_center(
        color_threshold_img, left_line.best_fit, right_line.best_fit, xm_per_pix
    )

    lanes_img = draw_lane(
        new_img, color_threshold_img, left_line.best_fit, right_line.best_fit, M_inv
    )

    result = draw_data(
        lanes_img, (radius_left + radius_right) / 2, distance_from_center
    )

    if fname is not None:
        # save images
        undist_img = cv2.cvtColor(undist_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(camera_calibration_undistorted_output_dir + fname, undist_img)
        warped_img = cv2.cvtColor(warped_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(perspective_transform_output_dir + fname, warped_img)
        cv2.imwrite(
            color_threshold_output_dir + fname, color_threshold_img * 255
        )  # multiply with 255 to see pixels
        lanes_img = cv2.cvtColor(lanes_img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(lane_lines_img_output_dir + fname, lanes_img)

    return result

<IPython.core.display.Javascript object>

# Test on images

In [20]:
# left_line = Line()
# right_line = Line()

# # Apply lane finding pipeline on test images
# for fname in os.listdir(test_images_input_dir):
#     test_img = cv2.imread(test_images_input_dir + fname)
#     test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

#     lane_finding_pipeline(test_img, left_line, right_line, fname=fname)

<IPython.core.display.Javascript object>

# Test on videos

In [21]:
left_line = Line()
right_line = Line()

project_video_output = output_videos_dir + "project_video_output.mp4"

project_video_input = VideoFileClip("./test_videos/project_video.mp4").fl_image(
    lambda image: lane_finding_pipeline(image, left_line, right_line)
)

%time project_video_input.write_videofile(project_video_output, audio=False)

t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video ./output_videos/project_video_output.mp4.
Moviepy - Writing video ./output_videos/project_video_output.mp4



Moviepy - Done !
Moviepy - video ready ./output_videos/project_video_output.mp4
CPU times: user 2min 35s, sys: 6.25 s, total: 2min 41s
Wall time: 1min 41s


<IPython.core.display.Javascript object>

In [22]:
left_line = Line()
right_line = Line()

challenge_video_output = output_videos_dir + "challenge_video_output.mp4"

challenge_video_input = VideoFileClip("./test_videos/challenge_video.mp4").fl_image(
    lambda image: lane_finding_pipeline(image, left_line, right_line)
)

%time challenge_video_input.write_videofile(challenge_video_output, audio=False)

t:   1%|          | 3/485 [00:00<00:28, 17.07it/s, now=None]

Moviepy - Building video ./output_videos/challenge_video_output.mp4.
Moviepy - Writing video ./output_videos/challenge_video_output.mp4



Moviepy - Done !
Moviepy - video ready ./output_videos/challenge_video_output.mp4
CPU times: user 56.9 s, sys: 2.82 s, total: 59.8 s
Wall time: 36.8 s


<IPython.core.display.Javascript object>

In [23]:
# left_line = Line()
# right_line = Line()

# harder_challenge_video_output = output_videos_dir + "harder_challenge_video_output.mp4"

# harder_challenge_video_input = VideoFileClip(
#     "./test_videos/harder_challenge_video.mp4"
# ).fl_image(lambda image: lane_finding_pipeline(image, left_line, right_line))

# %time harder_challenge_video_input.write_videofile(harder_challenge_video_output, audio=False)

<IPython.core.display.Javascript object>